In [1]:
!pip install easyfsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 2.7 MB/s eta 0:00:00


In [3]:
print("hi")
#from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path
import random
from statistics import mean
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import torch
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split #to do
from sklearn.preprocessing import OrdinalEncoder
#import seaborn as sns #vis
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import torchvision
from tqdm import tqdm
import random
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data_utils
random_seed = 2023
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


hi


In [4]:
df = pd.read_csv('/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv')
print(df.shape)
df=df.drop(["nameOrig","nameDest"],axis=1)
df = pd.get_dummies(df,columns=['type'])


(6362620, 11)


In [4]:
df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,0,0,1,0
1,1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,0,0,1,0
2,1,181.00,181.0,0.00,0.0,0.0,1,0,0,0,0,0,1
3,1,181.00,181.0,0.00,21182.0,0.0,1,0,0,1,0,0,0
4,1,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,0,0,1,0


In [5]:
df[(df["type_DEBIT"]==1 )& (df["isFraud"]==1)]
df[(df["type_CASH_IN"]==1 )& (df["isFraud"]==1)]

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER


In [6]:
x=df.drop(["isFraud","isFlaggedFraud","type_DEBIT","type_PAYMENT","type_CASH_IN"],axis=1)
x.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_OUT,type_TRANSFER
0,1,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,181.00,181.0,0.00,0.0,0.0,0,1
3,1,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [7]:
y=df["isFraud"]
y[y==1].count(),y[y==0].count(),(y[y==1].count()/y[y==0].count())*100
#why is it6354407 less?

(8213, 6354407, 0.129248881917699)

In [8]:
scaler = StandardScaler()

# Fit only to the training data

X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.4, shuffle = True, random_state = 8)
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
# Use the same function above for the validation set
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))
print(y_train[y_train==1].count())
print(y_test[y_test==1].count())

#x = df.values #returns a numpy array
X_train = torch.FloatTensor(X_train)
y_train = torch.tensor(y_train.values)
X_test= torch.FloatTensor(X_test)
y_test = torch.tensor(y_test.values)
X_train=X_train.reshape(X_train.shape[0],1,1,X_train.shape[1]).expand(-1,3,-1,-1)
X_test=X_test.reshape(X_test.shape[0],1,1,X_test.shape[1]).expand(-1,3,-1,-1)


X_train shape: (3817572, 8)
y_train shape: (3817572,)
X_test shape: (2545048, 8)
y_test shape: (2545048,)
4972
3241


In [9]:
train_set = data_utils.TensorDataset(X_train, y_train)
train_set.get_labels = lambda:  [ y.item() for y in y_train]

test_set = data_utils.TensorDataset(X_test, y_test)
test_set.get_labels = lambda:  [ y.item() for y in y_test]

In [10]:
N_WAY = 2  # Number of classes in a task
N_SHOT = 20  # Number of images per class in the support set
N_QUERY = 20  # Number of images per class in the query set
N_EVALUATION_TASKS = 100
from easyfsl.samplers import TaskSampler
train_sampler = TaskSampler(
    train_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=200
)
test_sampler = TaskSampler(
    test_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=100
)

In [11]:
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    #num_workers=n_workers,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)
test_loader = DataLoader(
    test_sampler,
    batch_sampler=test_sampler,
    #num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
class PrototypicalNetworks(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images: torch.Tensor,
        support_labels: torch.Tensor,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        """
        # Extract the features of support and query images
        z_support = self.backbone.forward(support_images)
        z_query = self.backbone.forward(query_images)

        # Infer the number of different classes from the labels of the support set
        n_way = len(torch.unique(support_labels))
        # Prototype i is the mean of all instances of features corresponding to labels == i
        z_proto = torch.cat(
            [
                z_support[torch.nonzero(support_labels == label)].mean(0)
                for label in range(n_way)
            ]
        )

        # Compute the euclidean distance from queries to prototypes
        dists = torch.cdist(z_query, z_proto)

        # And here is the super complicated operation to transform those distances into classification scores!
        scores = -dists
        return scores

#from easyfsl.methods import RelationNetworks
from torchvision.models import  resnet18,ResNet18_Weights
convolutional_network = resnet18(pretrained=True)
convolutional_network.fc = nn.Flatten()
print(convolutional_network)

model = PrototypicalNetworks(convolutional_network).to(device)
model=torch.load("/kaggle/input/model-param/proto_199_epochs_loss_0.02374154838253162.pth")


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 202MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
from torch import optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def fit(
    support_images: torch.Tensor,
    support_labels: torch.Tensor,
    query_images: torch.Tensor,
    query_labels: torch.Tensor,
) -> float:
    optimizer.zero_grad()
    classification_scores = model(
        support_images.cuda(), support_labels.cuda(), query_images.cuda()
    )

    loss = criterion(classification_scores, query_labels.cuda())
    loss.backward()
    optimizer.step()

    return loss.item()

In [15]:
def training_epoch(
    model, data_loader: DataLoader, optimizer: optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            true_class_IDs ,
        ) in tqdm_train:
            #print("shapessssssssssssssssssssssssssssssss",support_images.shape, support_labels.shape, query_images.shape, query_labels.shape)
            loss_value = fit(support_images, support_labels, query_images, query_labels)
            all_loss.append(loss_value)            
            tqdm_train.set_postfix(loss=mean(all_loss))
    return mean(all_loss)

training_epoch(model,train_loader, optimizer)

Training: 100%|██████████| 200/200 [00:12<00:00, 15.46it/s, loss=0.0247]


0.024666641342046203

In [16]:
best_state = model.state_dict()
best_validation_accuracy = 0.6

In [17]:
from torch.optim.lr_scheduler import MultiStepLR
#from torch.utils.tensorboard import SummaryWriter


scheduler_milestones = [50, 150]
scheduler_gamma = 0.1
tb_logs_dir = Path(".")

train_scheduler = MultiStepLR(
    optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)


In [21]:
from easyfsl.utils import evaluate

print("I'm not dead")
for epoch in range(100):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(model, train_loader, optimizer)
    #tb_writer.add_scalar("Train/loss", average_loss, epoch)
    #tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)
    if (epoch+1)%50==0:
        torch.save(model,"proto_"+str(100+epoch)+"_epochs_loss_"+str(average_loss)+".pth")
    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    #train_scheduler.step()

I'm not dead
Epoch 0


Training: 100%|██████████| 200/200 [00:05<00:00, 33.58it/s, loss=0.044] 


Epoch 1


Training: 100%|██████████| 200/200 [00:05<00:00, 34.99it/s, loss=0.0448]


Epoch 2


...

Training: 100%|██████████| 200/200 [00:05<00:00, 35.30it/s, loss=0.0297]


Epoch 98


Training: 100%|██████████| 200/200 [00:05<00:00, 33.39it/s, loss=0.021] 


Epoch 99


Training: 100%|██████████| 200/200 [00:05<00:00, 35.28it/s, loss=0.0237]


In [31]:
torch.save(model,"proto_100_epochs_loss_.02.pth")

13 loss is going down

In [18]:
i=3
n=4972
#read the data and drop the unuseful stuff
dff=pd.read_csv("/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv")
dff=dff.drop(["nameOrig","nameDest"],axis=1)
#category data
dff = pd.get_dummies(dff,columns=['type'])

#some types weren't really usefull :)
x=dff.drop(["isFlaggedFraud","type_DEBIT","isFraud","type_PAYMENT","type_CASH_IN"],axis=1)
y=dff["isFraud"]

X_tr, X_ts, y_tr, y_ts = train_test_split(x, y,test_size=0.4, shuffle = True, random_state = 8)

#srandrization
scaler = StandardScaler()

scaler.fit(X_tr)

X_tr = scaler.transform(X_tr)

#create a new dataframe that contain only the training data: extremly dump thing to do but will work on better version letter :|
d=pd.DataFrame(columns=dff.drop(["isFlaggedFraud","type_DEBIT","isFraud","type_PAYMENT","type_CASH_IN"],axis=1).columns)
d["isFraud"]=y_tr
col_names=dff.drop(["isFlaggedFraud","type_DEBIT","isFraud","type_PAYMENT","type_CASH_IN"],axis=1).columns
d[col_names]=X_tr

#fraud datset alone: only contain training samples
fraud=d[d["isFraud"]==1]
fraud_supprt_x=fraud.drop("isFraud",axis=1).values
fraud_supprt_x = torch.from_numpy(fraud_supprt_x).reshape(fraud_supprt_x.shape[0],1,1,fraud_supprt_x.shape[1]).expand(-1,3,-1,-1)
fraud_supprt_labels=fraud["isFraud"].values
print(fraud_supprt_x.shape)

#non fraud datset alone: only contain training samples
non_fraud=d[d["isFraud"]==0]
non_fraud_supprt_x=non_fraud.drop("isFraud",axis=1).values
non_fraud_supprt_x = torch.from_numpy(non_fraud_supprt_x).reshape(non_fraud_supprt_x.shape[0],1,1,non_fraud_supprt_x.shape[1]).expand(-1,3,-1,-1)
non_fraud_supprt_labels=non_fraud["isFraud"].values
print(non_fraud_supprt_x.shape)
import random
def chose_randomly_n_samples(x,y,n=2):
    l=list(range(x.shape[0]))
    idxs=random.sample(l, n)
    #print(x.shape,y.shape)
    return x[idxs],y[idxs]

non_fraud_supprt_x,non_fraud_supprt_labels=chose_randomly_n_samples(non_fraud_supprt_x, non_fraud_supprt_labels,n*i)
fraud_supprt_x,fraud_supprt_labels=chose_randomly_n_samples(fraud_supprt_x,fraud_supprt_labels,n)
supprt_labels=torch.zeros(n*(i+1))
supprt_labels[0:n]=torch.from_numpy(fraud_supprt_labels)
supprt_labels[n:n*(i+1)]=torch.from_numpy(non_fraud_supprt_labels)
print(supprt_labels.shape)

supprt_x=torch.zeros((n*(i+1),3,1,fraud_supprt_x.shape[-1]))
supprt_x[0:n]=fraud_supprt_x
supprt_x[n:n*(i+1)]=non_fraud_supprt_x
supprt_x.shape

torch.Size([4972, 3, 1, 8])
torch.Size([3812600, 3, 1, 8])
torch.Size([19888])


torch.Size([19888, 3, 1, 8])

In [ ]:
few_shot_classifier1=model
yt= y_test
#few_shot_classifier=torch.load("/kaggle/input/model-param/checkpoint1 (3).pth")
#few_shot_classifier.to(device)
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score,precision_score,confusion_matrix, ConfusionMatrixDisplay,average_precision_score,roc_auc_score, f1_score

def show_metrics(target,results):
    """
    show different metrics and display confusion matrix
    Args:
      target:the true value of the labels and should be numpy array or a list
      results: the predicted value of the labels and should be numpy array or a list
    """
    print("f1_score = ",f1_score(target, results))
    print("roc_auc_score = ",roc_auc_score(target, results))
    print("average_precision_score",average_precision_score(target, results))
    print("Precision : ",precision_score(target, results))
    print("Recall : ",recall_score(target, results))
    cm=confusion_matrix(target, results)
    tn, fp, fn, tp =cm.ravel()    
    print("tn",tn, "fp",fp, "fn",fn,"tp", tp )
    disp=ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()

def how_are_we_really_doing(model=few_shot_classifier1,train_loader=train_loader,X_test=X_test,target=yt
):
    #support_set(few_shot_classifier1, train_loader)
    results=np.zeros(y_test.shape[0])
    model.eval()
    torch.cuda.empty_cache()
    batch_size=64*32
    start =0
    end=1
    for i in tqdm(range(int(X_test.shape[0]/batch_size))):
        #support_images.cuda(), support_labels.cuda()
        pred=few_shot_classifier1(supprt_x.to(device), supprt_labels.to(device),X_test[batch_size*i:batch_size*(i+1)].to(device))
        results[batch_size*i:batch_size*(i+1)]=pred.data.argmax(1).cpu().numpy()
    show_metrics(target.numpy(), results)
how_are_we_really_doing()

## First 100 epochs
f1_score =  0.6613916686060043<br>
roc_auc_score =  0.9379509844648954<br>
average_precision_score 0.46571271569379485<br>
Precision :  0.530917242667663<br>
Recall :  0.8768898488120951<br>
tn 2539296 fp 2511 fn 399 tp 2842